<a href="https://colab.research.google.com/github/Xjkslhstxn/BT4222/blob/main/1.6%20Feature_selection_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/bt4222_notebook/

/content/drive/MyDrive/bt4222_notebook


In [ ]:
users_df = pd.read_csv('UserWithLabel_0.csv')

In [ ]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 95 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   UserId                               2000 non-null   int64  
 1   Q120                                 2000 non-null   object 
 2   MainBranch                           2000 non-null   object 
 3   Age                                  2000 non-null   object 
 4   Employment                           1999 non-null   object 
 5   RemoteWork                           1656 non-null   object 
 6   CodingActivities                     1655 non-null   object 
 7   EdLevel                              2000 non-null   object 
 8   LearnCode                            2000 non-null   object 
 9   LearnCodeOnline                      2000 non-null   object 
 10  LearnCodeCoursesCert                 880 non-null    object 
 11  YearsCode                     

In [ ]:
users_df.head(3)

,UserId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,processed_text,user_embeddings,top1_id,top1_score,top2_id,top2_score,bottom1_id,bottom1_score,bottom2_id,bottom2_score
0,41504,I agree,I am a developer by profession,25-34 years old,"Not employed, but looking for work",NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Online Courses or Certification;Other online r...,Formal documentation provided by the owner of ...,...,pythonrustsql dynamodbpostgresqlsqlite amazon ...,[-2.79997122e-02 -1.62469465e-02 -5.06044626e-...,10740,0.638643,3778,0.565087,1873,-0.007485,16138,0.016606
1,21778,I agree,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,Some college/university study without earning ...,Colleague;Online Courses or Certification;Othe...,Formal documentation provided by the owner of ...,...,kotlinphp mysqlpostgresqlredissqlite amazon we...,[ 1.64075214e-02 -3.10458057e-02 -6.21785223e-...,10497,0.635020,469,0.529684,15923,-0.021622,12372,-0.009894
2,47054,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Other online resources (e.g., videos, blogs, f...",Formal documentation provided by the owner of ...,...,javapythonsql bigquerypostgresqlredis amazon w...,[-1.91715509e-02 1.25910100e-02 -1.54270455e-...,673,0.631075,10740,0.589067,9414,0.025441,15923,0.028745


In [ ]:
jobs_df = pd.read_csv('embedded_jobs_IdEncoded.csv')

In [ ]:
print(jobs_df.head(3))

   JobId                                      advertiserurl  \
0      0  https://www.dice.com/jobs/detail/AUTOMATION-TE...   
1      1  https://www.dice.com/jobs/detail/Information-S...   
2      2  https://www.dice.com/jobs/detail/Business-Solu...   

                             company  \
0  Digital Intelligence Systems, LLC   
1  University of Chicago/IT Services   
2               Galaxy Systems, Inc.   

                            employmenttype_jobstatus  \
0  C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...   
1                                          Full Time   
2                                          Full Time   

                                      jobdescription joblocation_address  \
0  Looking for Selenium engineers...must have sol...         Atlanta, GA   
1  The University of Chicago has a rapidly growin...         Chicago, IL   
2  GalaxE.SolutionsEvery day, our solutions affec...      Schaumburg, IL   

                        jobtitle     postdate  \
0      

In [ ]:
print(jobs_df['employmenttype_jobstatus'].value_counts())

employmenttype_jobstatus
Full Time                                                                                                          6734
Contract W2                                                                                                        1102
Contract Corp-To-Corp, Contract Independent, Contract W2                                                            629
Full Time, Full Time                                                                                                617
Contract Corp-To-Corp, Contract Independent, Contract W2, C2H Corp-To-Corp, C2H Independent, C2H W2                 479
                                                                                                                   ... 
Contract W2, 3 Month+                                                                                                 1
Full Time, Contract W2, 6 mos w/ extensions                                                                           1
Contract Corp-T

In [ ]:
Job_extract = [
    'employmenttype_jobstatus',
    'postdate',
    'company'
]
extracted_df = jobs_df[Job_extract]
print(extracted_df.info())
print(extracted_df.head(3))
print(extracted_df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   employmenttype_jobstatus  21770 non-null  object
 1   postdate                  22000 non-null  object
 2   company                   21950 non-null  object
dtypes: object(3)
memory usage: 515.8+ KB
None
                            employmenttype_jobstatus     postdate  \
0  C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...   1 hour ago   
1                                          Full Time   1 week ago   
2                                          Full Time  2 weeks ago   

                             company  
0  Digital Intelligence Systems, LLC  
1  University of Chicago/IT Services  
2               Galaxy Systems, Inc.  
employmenttype_jobstatus    230
postdate                      0
company                      50
dtype: int64


In [ ]:
def concatenate_tech_columns(row):
    # Filter out None or NaN values before joining
    filtered_values = [str(value) for value in row if pd.notnull(value)]
    return ' '.join(filtered_values)

# Apply the function across the Tech_extract columns and create a new column for the merged data
extracted_df['Input_Merged'] = extracted_df[Job_extract].apply(concatenate_tech_columns, axis=1)

# Drop all columns in Tech_extract
extracted_df.drop(columns=Job_extract, inplace=True)
print(extracted_df.head(3))
print(extracted_df.isna().sum())

                                        Input_Merged
0  C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...
1  Full Time 1 week ago University of Chicago/IT ...
2         Full Time 2 weeks ago Galaxy Systems, Inc.
Input_Merged    0
dtype: int64


<ipython-input-18-1da9555e79ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_df['Input_Merged'] = extracted_df[Job_extract].apply(concatenate_tech_columns, axis=1)
<ipython-input-18-1da9555e79ef>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_df.drop(columns=Job_extract, inplace=True)


In [ ]:
jobs_df = jobs_df.join(extracted_df)
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   JobId                     22000 non-null  int64 
 1   advertiserurl             22000 non-null  object
 2   company                   21950 non-null  object
 3   employmenttype_jobstatus  21770 non-null  object
 4   jobdescription            22000 non-null  object
 5   joblocation_address       21997 non-null  object
 6   jobtitle                  22000 non-null  object
 7   postdate                  22000 non-null  object
 8   shift                     21643 non-null  object
 9   site_name                 3490 non-null   object
 10  skills                    21834 non-null  object
 11  Job_Features_Merged       22000 non-null  object
 12  processed_text            22000 non-null  object
 13  job_embeddings            22000 non-null  object
 14  Input_Merged          

In [ ]:
jobs_df['Input_Merged'].replace('', np.nan, inplace=True)
missing_values = jobs_df['Input_Merged'].isnull().sum()

print("Missing values after treating empty strings as NaN:", missing_values)

Missing values after treating empty strings as NaN: 0


In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 483.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(jobs_df.head(3))

   JobId                                      advertiserurl  \
0      0  https://www.dice.com/jobs/detail/AUTOMATION-TE...   
1      1  https://www.dice.com/jobs/detail/Information-S...   
2      2  https://www.dice.com/jobs/detail/Business-Solu...   

                             company  \
0  Digital Intelligence Systems, LLC   
1  University of Chicago/IT Services   
2               Galaxy Systems, Inc.   

                            employmenttype_jobstatus  \
0  C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...   
1                                          Full Time   
2                                          Full Time   

                                      jobdescription joblocation_address  \
0  Looking for Selenium engineers...must have sol...         Atlanta, GA   
1  The University of Chicago has a rapidly growin...         Chicago, IL   
2  GalaxE.SolutionsEvery day, our solutions affec...      Schaumburg, IL   

                        jobtitle     postdate  \
0      

In [ ]:
input_embeddings = model.encode(jobs_df['Input_Merged'].tolist())

In [ ]:
jobs_df['embeddings'] = list(input_embeddings)

In [ ]:
print(jobs_df.head(3))

   JobId                                      advertiserurl  \
0      0  https://www.dice.com/jobs/detail/AUTOMATION-TE...   
1      1  https://www.dice.com/jobs/detail/Information-S...   
2      2  https://www.dice.com/jobs/detail/Business-Solu...   

                             company  \
0  Digital Intelligence Systems, LLC   
1  University of Chicago/IT Services   
2               Galaxy Systems, Inc.   

                            employmenttype_jobstatus  \
0  C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...   
1                                          Full Time   
2                                          Full Time   

                                      jobdescription joblocation_address  \
0  Looking for Selenium engineers...must have sol...         Atlanta, GA   
1  The University of Chicago has a rapidly growin...         Chicago, IL   
2  GalaxE.SolutionsEvery day, our solutions affec...      Schaumburg, IL   

                        jobtitle     postdate  \
0      

In [ ]:
jobs_df.to_csv("JobsWithInput.csv", index = False)